# Gaussian mixtures
In this notebook we fit gaussian mixtures to the 25 stocks, using 2 components, then 3 components.

## Two-component models

In [29]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture


stocks = np.load('data/plr_stocks.npy')

model1 = GaussianMixture(n_components=2, tol=1e-5, n_init=10, max_iter=500)
model1.fit(stocks[0].reshape(-1, 1))

weights = model1.weights_        # shape (K,)
means = model1.means_.ravel()    # shape (K,)
variances = model1.covariances_.ravel()  # shape (K,) in 1D if covariance_type='full'

print("weights:", weights)
print("means:", means)
print("variances:", variances)
print('aic : ', model1.aic(stocks[0].reshape(-1, 1)), "\n")

model_test_init = GaussianMixture(n_components=2, tol=1e-5, weights_init=[0.49, 0.51], means_init=[[0], [0]])
model_test_init.fit(stocks[0].reshape(-1, 1))
print(model_test_init.weights_)
print(model_test_init.means_)
print(model_test_init.covariances_.ravel())


weights: [0.0637272 0.9362728]
means: [-0.67984417  0.08383665]
variances: [82.86802793  4.34503436]
aic :  37524.17321622367 

[0.06400394 0.93599606]
[[-0.67737361]
 [ 0.0838935 ]]
[82.59201867  4.3407766 ]


In [30]:
paramsfit2c = []
resultsfit2c = []
i = 1
number_inits = 10
import os
os.environ["OMP_NUM_THREADS"] = "1"


print('Fitting 2 component gaussian mixtures to the 25 stocks...')
print(f'Number of initializations : {number_inits} \n')

for stock in stocks:
    gmm2c = GaussianMixture(n_components = 2, tol = 1e-4, n_init = number_inits, init_params='random_from_data', random_state = 42)
    gmm2c.fit(stock.reshape(-1, 1))

    weights = gmm2c.weights_
    means = gmm2c.means_
    variances = gmm2c.covariances_.ravel()

    paramsfit2c.append({'stock': i, 'pi1': weights[0], 'pi2' : weights[1],
                        'mu1': means[0, 0], 'mu2' : means[1, 0],
                        'sigma1':np.sqrt(variances[0]), 'sigma2': np.sqrt(variances[1])})
    
    loglikelihood = gmm2c.score(stock.reshape(-1, 1)) * len(stock.reshape(-1, 1))
    aic = gmm2c.aic(stock.reshape(-1, 1))
    bic = gmm2c.bic(stock.reshape(-1, 1))

    resultsfit2c.append({'stock':i, 'log-likelihood':loglikelihood,
                         'aic':aic, 'bic':bic})

    i += 1

# saving results
results = pd.DataFrame(resultsfit2c)
fitted_params = pd.DataFrame(paramsfit2c)

print(results.head())
fitted_params.head()

results.to_csv('results/gmm2c_results.csv', index = False)
fitted_params.to_csv('results/gmm2c_fitted_params.csv', index = False)


Fitting 2 component gaussian mixtures to the 25 stocks...
Number of initializations : 10 

   stock  log-likelihood           aic           bic
0      1   -18756.928662  37523.857325  37558.793309
1      2   -16708.210247  33426.420493  33461.356477
2      3   -16265.908818  32541.817636  32576.753620
3      4   -16469.955592  32949.911185  32984.847169
4      5   -14564.103860  29138.207721  29173.143705


## Value at Risk estimation for 2-component model

In [31]:
from scipy.stats import Normal, Mixture

print(results.head())
#print(results.iloc[0]['mu'])

results_var2c = []

for i in range(results.shape[0]):

    pi1 = fitted_params.iloc[i]['pi1']
    pi2 = fitted_params.iloc[i]['pi2']
    mu1 = fitted_params.iloc[i]['mu1']
    mu2 = fitted_params.iloc[i]['mu2']
    sigma1 = fitted_params.iloc[i]['sigma1']
    sigma2 = fitted_params.iloc[i]['sigma2']

    alpha = 0.01
    dist1 = Normal(mu = mu1, sigma = sigma1)
    dist2 = Normal(mu = mu2, sigma = sigma2)
    mixt = Mixture(components = [dist1, dist2], weights = [pi1, pi2])

    empirical_var = -np.percentile(stocks[i], alpha * 100)
    var = -mixt.icdf(alpha)
    results_var2c.append({'stock': i + 1,'estimated-var':var, 'empirical-var':empirical_var})

    
results_var2c = pd.DataFrame(results_var2c)
results_var2c.head()
results_var2c.to_csv('results/ValueAtRiskResults/gmm2c_var.csv', index=False)


   stock  log-likelihood           aic           bic
0      1   -18756.928662  37523.857325  37558.793309
1      2   -16708.210247  33426.420493  33461.356477
2      3   -16265.908818  32541.817636  32576.753620
3      4   -16469.955592  32949.911185  32984.847169
4      5   -14564.103860  29138.207721  29173.143705


## Three component models

In [32]:
paramsfit3c = []
resultsfit3c = []
i = 1

print('Fitting 3 component gaussian mixtures to the 25 stocks...')
print(f'Number of initializations : {number_inits} \n')

for stock in stocks:
    gmm3c = GaussianMixture(n_components = 3, tol = 1e-4, n_init = number_inits, init_params='random_from_data', random_state=42)
    gmm3c.fit(stock.reshape(-1, 1))

    weights = gmm3c.weights_
    means = gmm3c.means_
    variances = gmm3c.covariances_.ravel()

    paramsfit3c.append({'stock': i, 'pi1': weights[0], 'pi2' : weights[1], 'pi3':weights[2],
                        'mu1': means[0, 0], 'mu2' : means[1, 0], 'mu3': means[2, 0],
                        'sigma1':np.sqrt(variances[0]), 'sigma2': np.sqrt(variances[1]), 
                        'sigma3': np.sqrt(variances[2])})
    
    loglikelihood = gmm3c.score(stock.reshape(-1, 1)) * len(stock.reshape(-1, 1))
    aic = gmm3c.aic(stock.reshape(-1, 1))
    bic = gmm3c.bic(stock.reshape(-1, 1))

    resultsfit3c.append({'stock':i, 'log-likelihood':loglikelihood,
                         'aic':aic, 'bic':bic})

    i += 1

# saving results
results = pd.DataFrame(resultsfit3c)
fitted_params = pd.DataFrame(paramsfit3c)

print(results.head())
print(fitted_params.head())

results.to_csv('results/gmm3c_results.csv', index = False)
fitted_params.to_csv('results/gmm3c_fitted_params.csv', index = False)


Fitting 3 component gaussian mixtures to the 25 stocks...
Number of initializations : 10 

   stock  log-likelihood           aic           bic
0      1   -18522.289615  37060.579231  37116.476805
1      2   -16586.670904  33189.341808  33245.239383
2      3   -16176.480734  32368.961468  32424.859043
3      4   -16445.697660  32907.395320  32963.292894
4      5   -14455.527983  28927.055966  28982.953540
   stock       pi1       pi2       pi3       mu1       mu2       mu3  \
0      1  0.659994  0.003441  0.336566  0.091330 -8.626583  0.013594   
1      2  0.049514  0.531635  0.418851 -0.319994  0.093557 -0.035094   
2      3  0.287556  0.019877  0.692567 -0.057723 -0.815180  0.069795   
3      4  0.689611  0.152602  0.157787  0.073809  0.043219 -0.174615   
4      5  0.009440  0.726279  0.264281 -0.891242  0.074207 -0.083245   

     sigma1     sigma2    sigma3  
0  1.541448  28.129486  3.789301  
1  6.282680   0.987567  2.430138  
2  2.688619   8.284984  1.212401  
3  1.620389   0.70

# Value at Risk estimation for three-component model

In [33]:
from scipy.stats import Normal, Mixture

print(results.head())

results_var3c = []

for i in range(fitted_params.shape[0]):

    pi1 = fitted_params.iloc[i]['pi1']
    pi2 = fitted_params.iloc[i]['pi2']
    pi3 = fitted_params.iloc[i]['pi3']
    mu1 = fitted_params.iloc[i]['mu1']
    mu2 = fitted_params.iloc[i]['mu2']
    mu3 = fitted_params.iloc[i]['mu3']
    sigma1 = fitted_params.iloc[i]['sigma1']
    sigma2 = fitted_params.iloc[i]['sigma2']
    sigma3 = fitted_params.iloc[i]['sigma3']

    alpha = 0.01
    dist1 = Normal(mu = mu1, sigma = sigma1)
    dist2 = Normal(mu = mu2, sigma = sigma2)
    dist3 = Normal(mu = mu3, sigma = sigma3)
    mixt = Mixture(components = [dist1, dist2, dist3], weights = [pi1, pi2, pi3])

    empirical_var = -np.percentile(stocks[i], alpha * 100)
    var = -mixt.icdf(alpha)
    print('Empirical var:', empirical_var)
    print('estimated var:',var, '\n')
    results_var3c.append({'stock': i + 1,'estimated-var':var, 'empirical-var':empirical_var})

    
results_var3c = pd.DataFrame(results_var3c)
results_var3c.head()
results_var3c.to_csv('results/ValueAtRiskResults/gmm3c_var.csv', index=False)


   stock  log-likelihood           aic           bic
0      1   -18522.289615  37060.579231  37116.476805
1      2   -16586.670904  33189.341808  33245.239383
2      3   -16176.480734  32368.961468  32424.859043
3      4   -16445.697660  32907.395320  32963.292894
4      5   -14455.527983  28927.055966  28982.953540
Empirical var: 7.460909595509617
estimated var: 7.4506947053412755 

Empirical var: 6.033780936051005
estimated var: 6.422672514831804 

Empirical var: 5.821528465112555
estimated var: 5.82775414617232 

Empirical var: 5.680408769439981
estimated var: 5.930811028240299 

Empirical var: 4.167985044887263
estimated var: 4.350167571104442 

Empirical var: 5.013481919836497
estimated var: 5.391479362684773 

Empirical var: 5.45530787598142
estimated var: 5.416066333911377 

Empirical var: 5.002146995149504
estimated var: 5.316356977894144 

Empirical var: 6.846430997702966
estimated var: 6.996976152106507 

Empirical var: 3.6742151568336774
estimated var: 3.76670580994393 

Emp